In [2]:
from federated_learning import Configuration
from federated_learning import LocalEnvironment

ImportError: cannot import name 'MNISTNet' from 'federated_learning.nets' (/home/jovyan/work/samples/federated_learning/nets/__init__.py)

In [ ]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
from torchvision import datasets, transforms
BATCH_SIZE = 128
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist_data', train=False, transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

RuntimeError: Dataset not found. You can use download=True to download it

In [ ]:
config = Configuration()
data = config.DATASET(config)
sim_env = LocalEnvironment(config, data)
sim_env.poison_clients()

In [ ]:
# print(torch.where(sim_env.clients[0].train_dataloader.dataset.dataset.targets == 5, 3, sim_env.clients[0].train_dataloader.dataset.dataset.targets))

In [ ]:
sim_env.clients[0].test()
for epoch in range(1, config.N_EPOCHS + 1):
    sim_env.clients[0].train(epoch)
    sim_env.clients[0].test()

In [ ]:
import numpy as np
import shap
examples = enumerate(test_loader)
batch_idx, (images, example_targets) = next(examples)
first_index=[]
for i in range(10):
    try:
        index = torch.where(example_targets==i)
        first_index.append(index[0][-1].item())
    except: 
        print("does not exist")
print(first_index)

background = images[:50]
test_images = images[first_index]

e = shap.DeepExplainer(sim_env.clients[0].net, background)
shap_values = e.shap_values(test_images)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
for i, idx in enumerate(first_index):
    plt.subplot(3,4,i+1)
    plt.tight_layout()
    plt.imshow(images[idx][0], cmap='gray', interpolation='none')
    plt.title("Ground Truth: {}".format(example_targets[idx]))
    plt.xticks([])
    plt.yticks([])
plt.show()

In [ ]:
output = sim_env.clients[0].net(images[first_index])
pred = output.data.max(1, keepdim=True)[1]
print(pred)

In [ ]:
shap_numpy = [np.swapaxes(np.swapaxes(s, 1, -1), 1, 2) for s in shap_values]
test_numpy = np.swapaxes(np.swapaxes(test_images.numpy(), 1, -1), 1, 2)

In [ ]:
shap.image_plot(shap_numpy, -test_numpy)